# Digit AI Trainer
#### How-to:
- Paste function into last cell
- Run all cells

In [29]:
import torch
from fastai.vision.all import *
import logging
import time
import numpy as np
from sklearn.metrics import accuracy_score
import torch.nn as nn
from collections import OrderedDict


def summary(model, input_size, batch_size=-1, device="cuda"):
# Copied from torchsummary
    def register_hook(module):

        def hook(module, input, output):
            class_name = str(module.__class__).split(".")[-1].split("'")[0]
            module_idx = len(summary)

            m_key = "%s-%i" % (class_name, module_idx + 1)
            summary[m_key] = OrderedDict()
            summary[m_key]["input_shape"] = list(input[0].size())
            summary[m_key]["input_shape"][0] = batch_size
            if isinstance(output, (list, tuple)):
                summary[m_key]["output_shape"] = [
                    [-1] + list(o.size())[1:] for o in output
                ]
            else:
                summary[m_key]["output_shape"] = list(output.size())
                summary[m_key]["output_shape"][0] = batch_size

            params = 0
            if hasattr(module, "weight") and hasattr(module.weight, "size"):
                params += torch.prod(torch.LongTensor(list(module.weight.size())))
                summary[m_key]["trainable"] = module.weight.requires_grad
            if hasattr(module, "bias") and hasattr(module.bias, "size"):
                params += torch.prod(torch.LongTensor(list(module.bias.size())))
            summary[m_key]["nb_params"] = params

        if (
            not isinstance(module, nn.Sequential)
            and not isinstance(module, nn.ModuleList)
            and not (module == model)
        ):
            hooks.append(module.register_forward_hook(hook))

    device = device.lower()
    assert device in [
        "cuda",
        "cpu",
    ], "Input device is not valid, please specify 'cuda' or 'cpu'"

    if device == "cuda" and torch.cuda.is_available():
        dtype = torch.cuda.FloatTensor
    else:
        dtype = torch.FloatTensor

    # multiple inputs to the network
    if isinstance(input_size, tuple):
        input_size = [input_size]

    # batch_size of 2 for batchnorm
    x = [torch.rand(2, *in_size).type(dtype) for in_size in input_size]
    # print(type(x[0]))

    # create properties
    summary = OrderedDict()
    hooks = []

    # register hook
    model.apply(register_hook)

    # make a forward pass
    # print(x.shape)
    model(*x)

    # remove these hooks
    for h in hooks:
        h.remove()

    total_params = 0
    total_output = 0
    trainable_params = 0
    for layer in summary:
        # input_shape, output_shape, trainable, nb_params
        line_new = "{:>20}  {:>25} {:>15}".format(
            layer,
            str(summary[layer]["output_shape"]),
            "{0:,}".format(summary[layer]["nb_params"]),
        )
        total_params += summary[layer]["nb_params"]
        total_output += np.prod(summary[layer]["output_shape"])
        if "trainable" in summary[layer]:
            if summary[layer]["trainable"] == True:
                trainable_params += summary[layer]["nb_params"]

    # assume 4 bytes/number (float on cuda).
    total_input_size = abs(np.prod(input_size) * batch_size * 4. )
    total_output_size = abs(2. * total_output * 4. )  # x2 for gradients
    total_params_size = abs(total_params.numpy() * 4.)
    estimated_size = total_params_size + total_output_size + total_input_size

    # total_size_str = "Estimated Total Size (MB): %0.2f" % total_size
    return estimated_size

def format_elapsed_time(start_time, end_time):
    # Calculate the time difference in seconds
    delta_time = end_time - start_time

    # Convert to minutes and hours if necessary
    if delta_time >= 3600:  # If delta_time is greater than or equal to 1 hour
        hours = int(delta_time // 3600)
        delta_time %= 3600
    else:
        hours = 0

    if delta_time >= 60:  # If delta_time is greater than or equal to 1 minute
        minutes = int(delta_time // 60)
        delta_time %= 60
    else:
        minutes = 0

    seconds = delta_time  # Remaining time in seconds
    formatted_seconds = f"{seconds:.2f}"  # Format seconds with 2 decimal places

    # Construct and return the formatted string
    return f"{hours} hours, {minutes} minutes, {formatted_seconds} seconds"

def save_confusion_matrix(interp, pic_path, cmap:str="Blues"):
    title = "Confusion Matrix"
    "Plot the confusion matrix, with `title` and using `cmap`."
    # This function is mainly copied from the sklearn docs
    cm = interp.confusion_matrix()
    fig = plt.figure()
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    tick_marks = np.arange(len(interp.vocab))
    plt.xticks(tick_marks, interp.vocab, rotation=90)
    plt.yticks(tick_marks, interp.vocab, rotation=0)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        coeff = f'{cm[i, j]}'
        plt.text(j, i, coeff, horizontalalignment="center", verticalalignment="center", color="white"
                    if cm[i, j] > thresh else "black")

    ax = fig.gca()
    ax.set_ylim(len(interp.vocab)-.5,-.5)

    plt.tight_layout()
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.grid(False)
    fig.savefig(pic_path)

def train_model(digit_folder="Projects/digit/digits", batch_size=32, epochs=12, seed=69, lr=3e-3, model_output="Projects/digit/models.pkl", save_conf=False, conf_out='None', base_model="resnet18"):
    # Logging
    torch.cuda.empty_cache()
    log_name = model_output.replace(".pkl", ".txt")
    logging.getLogger().handlers = []
    logging.basicConfig(level=logging.INFO)
    file_handler = logging.FileHandler(log_name)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s', '%H:%M:%S')
    file_handler.setFormatter(formatter)
    logging.getLogger().addHandler(file_handler)
    with open(log_name, 'w'):
        pass
    logging.info('Logger setup')
    
    # Setup device
    available = torch.cuda.is_available()
    logging.info(f'CUDA: {available}')
    device = torch.device('cuda' if available else 'cpu')
    logging.info(f"Device: {torch.cuda.get_device_name(device)}")

    model_map = {

        'alexnet': models.alexnet,
        'densenet121': models.densenet121,
        'densenet161': models.densenet161,
        'densenet201': models.densenet201,
        'efficientnet_b0': models.efficientnet_b0,
        'efficientnet_b1': models.efficientnet_b1,
        'googlenet': models.googlenet,
        'inception_v3': models.inception_v3,
        'mnasnet0_5': models.mnasnet0_5,
        'mnasnet1_0': models.mnasnet1_0,
        'mnasnet0_75': models.mnasnet0_75,
        'mnasnet1_3': models.mnasnet1_3,
        'mobilenetV2': models.mobilenet_v2,
        'mobilenet_v3_small': models.mobilenet_v3_small,
        'resnet18': models.resnet18,
        'resnet34': models.resnet34,
        'resnet50': models.resnet50,
        'squeezenet1_0': models.squeezenet1_0,
        'squeezenet1_1': models.squeezenet1_1,
        'vgg13_bn': models.vgg13_bn,
        'vgg16': models.vgg16,
        'vgg19_bn': models.vgg19_bn,
    }


    arch = model_map[base_model]

    logging.info(f'Base model architecture: {base_model}')

    # Load data, set seed, augment training photos
    set_seed(seed)

    logging.info(f'Seed: {seed}')

    fingers = DataBlock(
        blocks=(ImageBlock, CategoryBlock),
        get_items=get_image_files,
        splitter=GrandparentSplitter(),
        get_y=parent_label,
        batch_tfms=aug_transforms(mult = 1.5,max_zoom=1.))
    
    logging.info(f'DataBlock created')

    dls = fingers.dataloaders(digit_folder, batch_size=batch_size)

    logging.info(f'DataLoader created')

    # Create classifier
    learn = vision_learner(dls, arch, metrics=error_rate, lr=lr)
    learn.model.to(device)

    logging.info(f'Learner created on {device}')

    logging.info(f'Training started: \n\tNumber of epochs = {epochs} \n\tBatch size = {batch_size} \n\tLearning rate = {lr}')
    time1 = time.time()

    # Check estimated memory usage
    device_properties = torch.cuda.get_device_properties(0)
    device_memory = device_properties.total_memory
    logging.info(f"Device memory (MB): {device_memory / 1024**2:.2f}")
    try:
        est_mem = summary(arch().cuda(), (3, 480, 640), batch_size=batch_size)
        if est_mem > device_memory:
            logging.info(f"Estimated Total Size (MB): {est_mem / 1024**2:.2f}")
            logging.warning("Estimated memory usage is larger than device's memory")
    except:
        logging.warning("Could not estimate estimated memory usage")


    # Train classifier
    learn.fine_tune(epochs)
    torch.cuda.empty_cache()

    time2 = time.time()

    dt = format_elapsed_time(time1, time2)

    logging.info(f'Training finished \n\tTime taken = {dt}')

    # Analyze classifier
    if save_conf:
        interp = ClassificationInterpretation.from_learner(learn)
        save_confusion_matrix(interp, conf_out)
        logging.info(f'Confusion matrix: \n{interp.confusion_matrix()}')

    # Accuracy score
    preds, targets = learn.get_preds(dl=dls.valid)
    acc = accuracy_score(targets.numpy(), np.argmax(preds, axis=1))
    logging.info(f'Validation accuracy = {acc}')

    # Export model
    # Check if file name ends in .pkl
    if not model_output.endswith(".pkl"):
            model_output += ".pkl"

    learn.export(fname=model_output)

    # Close logging
    logging.info("Exiting logger")
    file_handler.close()

##### Paste code into cell below

In [ ]:
# Example: train_model(save_conf=False, epochs=0, base_model="vgg16", batch_size=32)
train_model(save_conf=False, epochs=0, base_model="resnet18", batch_size=32)